In [1]:
import numpy as np
from scipy import linalg, special
import seaborn as sns
import pandas as pd
import os
from inspect import getmembers
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import dill
from equilibrator_api import ComponentContribution, Q_, Reaction
import requests
import xmltodict
import pint
import cvxpy as cp
import itertools
import copy

sns.set(style='darkgrid', palette='viridis', context='talk')

os.chdir(os.path.expanduser('~/vivarium-ecoli'))

A + B -E1> C -E2> D

B -E3> F

A -E4> G

# Testing eQuilibrator

In [2]:
cc = ComponentContribution()

# optional: changing the aqueous environment parameters
cc.p_h = Q_(7.4)
cc.p_mg = Q_(3.0)
cc.ionic_strength = Q_("0.25M")
cc.temperature = Q_("298.15K")



In [3]:
from equilibrator_api import Reaction
compound_ids = ["WATER", "ADP", "ATP", "Pi"]
compound_dict = {cid : cc.get_compound(f"metacyc.compound:{cid}") for cid in compound_ids}
atpase_reaction = Reaction({
    compound_dict["ATP"]: -1,
    compound_dict["WATER"]: -1,
    compound_dict["ADP"]: 1,
    compound_dict["Pi"]: 1,
})

In [4]:
standard_dg_prime = cc.standard_dg_prime(atpase_reaction)
standard_dg_prime

<Measurement(-29.144472212600157, 0.3042778553577678, kilojoule / mole)>

In [5]:
cytoplasmic_p_h = Q_(7.5)
cytoplasmic_ionic_strength = Q_("250 mM")
periplasmic_p_h = Q_(7.0)
periplasmic_ionic_strength = Q_("200 mM")
e_potential_difference = Q_("0.15 V")
cytoplasmic_reaction = "bigg.metabolite:pep = bigg.metabolite:g6p + bigg.metabolite:pyr"
periplasmic_reaction = "bigg.metabolite:glc__D = "

cc = ComponentContribution()
cc.p_h = cytoplasmic_p_h
cc.ionic_strength = cytoplasmic_ionic_strength
standard_dg_prime = cc.multicompartmental_standard_dg_prime(
    cc.parse_reaction_formula(cytoplasmic_reaction),
    cc.parse_reaction_formula(periplasmic_reaction),
    e_potential_difference=e_potential_difference,
    p_h_outer=periplasmic_p_h,
    ionic_strength_outer=periplasmic_ionic_strength,
)

print(standard_dg_prime)


(-44.8 +/- 0.6) kilojoule / mole


In [6]:
cc.get_compound_by_inchi("WQZGKKKJIJFFOK-GASJEMHNSA-N")

# Toy example with fake data

In [7]:
s = requests.Session() # create session
# Post login credentials to session:
s.post('https://websvc.biocyc.org/credentials/login/', data={'email':'cellulararchitect@protonmail.com', 'password':'Cellman0451'})
# Issue web service request:
r = s.get('https://websvc.biocyc.org/getxml?id=ECOLI:6PFRUCTPHOS-RXN&detail=low&fmt=json')

In [8]:
rxn = xmltodict.parse(r.content)
rxn['ptools-xml']['Reaction']['right']

[{'Compound': {'@resource': 'getxml?ECOLI:PROTON',
   '@orgid': 'ECOLI',
   '@frameid': 'PROTON'}},
 {'Compound': {'@resource': 'getxml?ECOLI:ADP',
   '@orgid': 'ECOLI',
   '@frameid': 'ADP'}},
 {'Compound': {'@resource': 'getxml?ECOLI:FRUCTOSE-16-DIPHOSPHATE',
   '@orgid': 'ECOLI',
   '@frameid': 'FRUCTOSE-16-DIPHOSPHATE'}}]

In [9]:
rxn['ptools-xml']['Reaction']['left'][0]['Compound']['@frameid']

'FRUCTOSE-6P'

In [10]:
r = s.get(f'https://websvc.biocyc.org/getxml?id=ECOLI:F16ALDOLASE-RXN&detail=low&fmt=json')
rxn = xmltodict.parse(r.content)
rxn['ptools-xml']['Reaction']['left']

{'Compound': {'@resource': 'getxml?ECOLI:FRUCTOSE-16-DIPHOSPHATE',
  '@orgid': 'ECOLI',
  '@frameid': 'FRUCTOSE-16-DIPHOSPHATE'}}

In [11]:
rxns_names = ['6PFRUCTPHOS-RXN', 'F16ALDOLASE-RXN', '2TRANSKETO-RXN', 'TRIOSEPISOMERIZATION-RXN']

rxns_dict = {}
stoich_dict = {}

for name in rxns_names:
    r = s.get(f'https://websvc.biocyc.org/getxml?id=ECOLI:{name}&detail=low&fmt=json')
    rxn = xmltodict.parse(r.content)

    rxn_dict = {}
    stoich_loop_dict = {}
    left = rxn['ptools-xml']['Reaction']['left']
    right = rxn['ptools-xml']['Reaction']['right']
    
    if type(left) is dict:
        left = [left]
    
    if type(right) is dict:
        right = [right]
    
    for mol in left:
        if type(mol) is dict:
            cid = mol['Compound']['@frameid']
            mol_cc = cc.get_compound(f"metacyc.compound:{cid}")
            rxn_dict[mol_cc] = -1
            stoich_loop_dict[cid] = -1

    for mol in right:
        if type(mol) is dict:
            cid = mol['Compound']['@frameid']
            mol_cc = cc.get_compound(f"metacyc.compound:{cid}")
            rxn_dict[mol_cc] =  1
            stoich_loop_dict[cid] = 1
    
    rxns_dict[name] = Reaction(rxn_dict)
    stoich_dict[name] = stoich_loop_dict
    
rxns_dict

{'6PFRUCTPHOS-RXN': <equilibrator_api.phased_reaction.PhasedReaction at 0x13a15f160>,
 'F16ALDOLASE-RXN': <equilibrator_api.phased_reaction.PhasedReaction at 0x15faed730>,
 '2TRANSKETO-RXN': <equilibrator_api.phased_reaction.PhasedReaction at 0x13a160bb0>,
 'TRIOSEPISOMERIZATION-RXN': <equilibrator_api.phased_reaction.PhasedReaction at 0x1744cdbe0>}

In [12]:
list(rxns_dict.values())

In [13]:
(standard_dg_prime, dg_uncertainty) = cc.standard_dg_prime_multi(list(rxns_dict.values()), uncertainty_representation="cov")

In [14]:
standard_dg_prime

Magnitude,[-21.05010908198642 22.047924530096907 -5.946115510564027 -5.617742386692498]
Units,kilojoule/mole


In [15]:
dg_uncertainty

Magnitude,[[1.5398212990074922 -2.2167471180569773 0.6541085596545129 -0.004664260811860303] [-2.2167471180569773 5.286946034707553 -2.4322877177623594 -0.1291182736017488] [0.6541085596545129 -2.4322877177623594 5.547270812427002 -0.08893998540652082] [-0.004664260811860303 -0.1291182736017488 -0.08893998540652082 0.3010346102557717]]
Units,kilojoule2/mole2


In [16]:
R = 0.008314 # kJ/mol*K
T = 298.15 # K

In [17]:
dG = standard_dg_prime._magnitude

keq = np.exp(-dG/(R*T))
keq

array([4.87556287e+03, 1.37137368e-04, 1.10096201e+01, 9.64363243e+00])

In [18]:
vars(standard_dg_prime)

{'_magnitude': array([-21.05010908,  22.04792453,  -5.94611551,  -5.61774239]),
 '_units': <UnitsContainer({'kilojoule': 1, 'mole': -1})>,
 '_Quantity__used': False,
 '_Quantity__handling': None,
 '_dimensionality': <UnitsContainer({'[length]': 2, '[mass]': 1, '[substance]': -1, '[time]': -2})>}

# Making example data for toy problem that fits physical constraints

Stochiometric matrix:

In [743]:
Sd = pd.DataFrame(stoich_dict, dtype=np.int8).fillna(0).astype(np.int8)
# Sd = Sd.iloc[0:7, 0:2]

n_met = len(Sd.index)
n_rxn = len(Sd.columns)

Sd

,6PFRUCTPHOS-RXN,F16ALDOLASE-RXN,2TRANSKETO-RXN,TRIOSEPISOMERIZATION-RXN
FRUCTOSE-6P,-1,0,1,0
ATP,-1,0,0,0
PROTON,1,0,0,0
ADP,1,0,0,0
FRUCTOSE-16-DIPHOSPHATE,1,-1,0,0
DIHYDROXY-ACETONE-PHOSPHATE,0,1,0,1
GAP,0,1,1,-1
ERYTHROSE-4P,0,0,-1,0
XYLULOSE-5-PHOSPHATE,0,0,-1,0


In [744]:
dG = standard_dg_prime._magnitude

keq = np.exp(-dG/(R*T))
keq

K_eq = np.log(keq)
vE = np.array([100, 20, -30, -10])

K_eq[vE < 0] = 1/K_eq[vE < 0] 

lvE = np.log(np.abs(vE))

In [745]:
# set up variables

S = np.array(Sd)
S = np.multiply(S, vE/np.abs(vE)).astype(np.int8)
S[S == -0] = 0
S_s = -np.copy(S)
S_p = np.copy(S) #reverse neg sign
S_s[S > 0] = 0
S_p[S < 0] = 0

S_s_nz = np.array(S_s.nonzero())
S_p_nz = np.array(S_p.nonzero())

# first coordinate, e.g. metabolites w nonzero substrate/product coeff across all reactions. also works as substrate indices. 
met_s_nz = S_s_nz[0, :]
met_p_nz = S_p_nz[0, :]

# second coordinate, e.g. reactions indices for those concentrations. works to index substrates as well. 
rxn_s_nz = S_s_nz[1, :]   
rxn_p_nz = S_p_nz[1, :]

# one dim is always 2
n_Km_s = np.max(met_s_nz.shape) 
n_Km_p = np.max(met_p_nz.shape)

c = cp.Variable(n_met)
Km_s = cp.Variable(n_Km_s)
Km_p = cp.Variable(n_Km_p)

cfwd = cp.Variable(n_rxn)
crev = cp.Variable(n_rxn)

# define Km positions by nonzero S matrix concentrations
y_s = c[met_s_nz] - Km_s
y_p = c[met_p_nz] - Km_p

# index 
met_s_nz

array([0, 0, 1, 4, 5, 6])

In [746]:
met_s_nz

array([0, 0, 1, 4, 5, 6])

In [747]:
rxn_s_nz

array([0, 2, 0, 1, 3, 2])

In [748]:
# number of saturation terms for sub, prod
n_alpha = np.sum(np.power(2, S_s.sum(axis=0)) - 1)
n_beta = np.sum(np.power(2, S_p.sum(axis=0)) - 1)

# saturation matrix setup
C_alpha = np.zeros([n_alpha, len(met_s_nz)])
C_beta = np.zeros([n_beta, len(met_p_nz)])

# to separate different reactions saturation terms. 
d_alpha = np.zeros(n_alpha, dtype=np.int8)
d_beta = np.zeros(n_beta, dtype=np.int8)


idx = 0

for i in range(n_rxn):
    
    # pick one reaction at a time (get substrate indicies)
    idx_cur_rxn = rxn_s_nz == i
    
    # generates all binary permutations minus the first one since that would result in -1
    sat_perm = np.array(list(itertools.product([0, 1], repeat=sum(idx_cur_rxn))))
    sat_perm = sat_perm[1:, :]
    
    r, _ = sat_perm.shape
    
    # replace zeros with saturation matrix
    C_alpha[idx:(idx+r), idx_cur_rxn] = sat_perm
    d_alpha[idx:(idx+r)] = i
        
    idx += r # add row # 

idx = 0
    
for i in range(n_rxn):
    idx_cur_rxn = rxn_p_nz == i
    
    sat_perm = np.array(list(itertools.product([0, 1], repeat=sum(idx_cur_rxn))))
    sat_perm = sat_perm[1:, :]
    
    r, _ = sat_perm.shape
    
    C_beta[idx:(idx+r), idx_cur_rxn] = sat_perm
    d_beta[idx:(idx+r)] = i
        
    idx += r # add row # 

In [749]:
C_alpha

array([[0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.]])

In [755]:
-S.T[i, S_s_idx]

array([1, 1], dtype=int8)

In [756]:
n_lse_terms = np.max(np.power(2, S_s.sum(axis=0)) +  np.power(2, S_p.sum(axis=0)) - 2)
LSE_expr = []

for i in range(n_rxn):
    # sum terms are separate in logsumexp. one per saturation term (row in C_alpha, C_beta)
    n_term_s = np.sum(d_alpha == i) 
    n_term_p = np.sum(d_beta == i)
    n_term = n_term_s + n_term_p
    
    Km_s_idx = np.nonzero(S_s_nz[1, :] == i)
    S_s_idx = S_s_nz[0, S_s_nz[1, :] == i] # negate -1 entries
    
    Km_p_idx = np.nonzero(S_p_nz[1, :] == i)
    S_p_idx = S_p_nz[0, S_p_nz[1, :] == i]
    
    LSE_expr.append(cp.hstack( [ (C_alpha @ y_s)[d_alpha == i] - cp.multiply(np.ones(n_term_s), -S.T[i, S_s_idx] @ y_s[Km_s_idx]) - cfwd[i],  
                                 (C_beta @ y_p)[d_beta == i] - cp.multiply(np.ones(n_term_p), -S.T[i, S_s_idx] @ y_s[Km_s_idx]) - cfwd[i],
                                 0 - cp.multiply(np.ones(1), -S.T[i, S_s_idx] @ y_s[Km_s_idx])  - cfwd[i],
                                 #-1*np.ones(n_lse_terms - n_term + 1) 
                               ]
                             )
                   )  # remove +1 here, could also have cfwd outside objec. 
    
#LSE_expr = cp.vstack(LSE_expr)
LSE_expr

[Expression(AFFINE, UNKNOWN, (11,)),
 Expression(AFFINE, UNKNOWN, (5,)),
 Expression(AFFINE, UNKNOWN, (7,)),
 Expression(AFFINE, UNKNOWN, (3,))]

In [757]:
l = 0.000#1
e = 0.00001
f = 0.000#1
reg = cp.sum(cp.huber(cfwd - 2*lvE, 1))
reg2 = cp.norm1(cp.hstack([cfwd, crev, c, -Km_s, -Km_p])) # regularization
reg3 = cp.sum(cp.huber(cp.hstack([y_s, y_p]), 1))
#reg3 = cp.norm1(cp.hstack([y_s, y_p])) # take a look at this

# e1 = LSE_expr - cfwd

# loss = cp.sum(cp.log_sum_exp(cp.vstack(LSE_expr), axis=1)  + )
loss = 0
for i in range(n_rxn):
    loss += cp.norm2(cp.pos(cp.log_sum_exp(LSE_expr[i])  - (-lvE[i])   ))
#loss = cp.norm2(cp.pos(cp.log_sum_exp(cp.vstack(LSE_expr), axis=1)  - (-lvE)   )) # use + lvE as it has negative exponent
loss += l * reg 
loss += e * reg2
loss += f * reg3
# 

In [758]:
haldane = []
fwd_flux = []

for i, r in enumerate(S.T):
    Km_s_idx = np.nonzero(S_s_nz[1, :] == i)
    S_s_idx = S_s_nz[0, S_s_nz[1, :] == i] # negate -1 entries
    
    Km_p_idx = np.nonzero(S_p_nz[1, :] == i)
    S_p_idx = S_p_nz[0, S_p_nz[1, :] == i]
    
    haldane.append(K_eq[i] == cfwd[i] - crev[i] + r[S_p_idx] @ Km_p[Km_p_idx] - (-r[S_s_idx]) @ Km_s[Km_s_idx])  # add minus since s matrix has minus
    fwd_flux.append(cfwd[i] + (-r[S_s_idx]) @ y_s[Km_s_idx] - (crev[i] + r[S_p_idx] @ y_p[Km_p_idx]) >= 0)  # add minus since s matrix has minus
    
haldane

[Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONPOSITIVE, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ()))]

In [759]:
constr = [cp.hstack([cfwd, crev, c, Km_s, Km_p]) >= -5,
          cp.hstack([cfwd, crev, c, Km_s, Km_p]) <= 10, 
          ]

constr.extend(haldane)
constr.extend(fwd_flux)
constr.extend([S.T @ c <= K_eq])

In [760]:
p = cp.Problem(cp.Minimize(loss), constr)
p.solve()

0.000526187244592638

# Checking correctness of result

In [761]:
cfwd.value

array([8.49074566, 2.99831788, 4.54453343, 3.20901502])

In [762]:
crev.value

array([4.77284994e-07, 9.93848148e+00, 4.12392751e+00, 2.76774693e+00])

In [763]:
c.value

array([ 5.66058630e+00,  3.14112784e-02, -1.61386266e-05, -1.61386266e-05,
        8.15060611e+00, -5.08545246e-05, -7.43883225e-01, -2.48525949e-07,
       -2.48525949e-07])

In [764]:
Km_s.value

array([-3.02878606e-04,  6.04630178e-04, -6.13887710e-04,  2.13979733e-01,
       -5.01987226e-06, -1.03328878e-05])

In [765]:
Km_p.value

array([ 1.51530412e-05,  1.51530412e-05,  2.98575868e-04, -6.72354663e-01,
       -1.06802945e+00, -2.48411197e-05, -1.56560441e-03, -1.56560441e-03])

## Checking Haldane and fwd/rev flux ratios are satisfied

In [766]:
for i, r in enumerate(S.T):
    Km_s_idx = np.nonzero(S_s_nz[1, :] == i)
    S_s_idx = S_s_nz[0, S_s_nz[1, :] == i] # negate -1 entries
    
    Km_p_idx = np.nonzero(S_p_nz[1, :] == i)
    S_p_idx = S_p_nz[0, S_p_nz[1, :] == i]
    
    print("haldane", cfwd.value[i] - crev.value[i] + r[S_p_idx] @ Km_p.value[Km_p_idx] - (-r[S_s_idx]) @ Km_s.value[Km_s_idx] - K_eq[i] )
    # print(cfwd.value[i], (-r[S_s_idx]), y_s.value[Km_s_idx], crev.value[i], r[S_p_idx],  y_p.value[Km_p_idx])
    print("forward", cfwd.value[i] + (-r[S_s_idx]) @ y_s.value[Km_s_idx] - (crev.value[i] + r[S_p_idx] @ y_p.value[Km_p_idx]))

haldane -3.6924596713561186e-10
forward 6.0334145816157
haldane 3.8674841107422253e-10
forward 1.2743160718642343e-05
haldane -1.8129109324860337e-11
forward 5.3335839912316585
haldane -1.9188595157260124e-11
forward 1.1850806384993802


In [767]:
y_s.value

array([ 5.66088918e+00,  5.65998167e+00,  3.20251661e-02,  7.93662638e+00,
       -4.58346523e-05, -7.43872892e-01])

In [768]:
y_p.value

array([-3.12916678e-05, -3.12916678e-05,  8.15030754e+00,  6.72303809e-01,
        3.24146225e-01, -7.43858384e-01,  1.56535589e-03,  1.56535589e-03])

## Checking that objective has been minimized.

In [769]:
LSE_expr = []

for i in range(n_rxn):
    # sum terms are separate in logsumexp. one per saturation term (row in C_alpha, C_beta)
    n_term_s = np.sum(d_alpha == i) 
    n_term_p = np.sum(d_beta == i)
    n_term = n_term_s + n_term_p
    
    Km_s_idx = np.nonzero(S_s_nz[1, :] == i)
    S_s_idx = S_s_nz[0, S_s_nz[1, :] == i] # negate -1 entries
    
    Km_p_idx = np.nonzero(S_p_nz[1, :] == i)
    S_p_idx = S_p_nz[0, S_p_nz[1, :] == i]
    
    
    LSE_expr.append(           [ (C_alpha @ y_s.value)[d_alpha == i] - np.multiply(np.ones(n_term_s), -S.T[i, S_s_idx] @ y_s.value[Km_s_idx]) - cfwd.value[i],  
                                 (C_beta @ y_p.value)[d_beta == i] - np.multiply(np.ones(n_term_p), -S.T[i, S_s_idx] @ y_s.value[Km_s_idx]) - cfwd.value[i],
                                 0 - np.multiply(np.ones(1), -S.T[i, S_s_idx] @ y_s.value[Km_s_idx])  - cfwd.value[i],
                                 #-1*np.ones(n_lse_terms - n_term + 1) 
                               ]
                   )
    

est = np.zeros(4)    

for i, rxn in enumerate(LSE_expr):
    s = 0
    
    for term in rxn:
        s += np.sum(np.exp(term))
        
    est[i] = np.log(s)
    

# est is obj. 
print(np.array([np.exp(-est),np.exp(lvE)]))

[[100.02798216  20.00137897  30.00295871  10.00050469]
 [100.          20.          30.          10.        ]]


## How closely does the objective match our target kcats?

In [770]:
print(np.array([np.exp(-est),np.exp(lvE)]))

[[100.02798216  20.00137897  30.00295871  10.00050469]
 [100.          20.          30.          10.        ]]


Remarkable. 

# Does it work like an enzyme?

Let's raise the concentration of met 1. 

In [771]:
# dG = standard_dg_prime._magnitude

# keq = np.exp(-dG/(R*T))
# keq

# K_eq = np.log(keq)

# # set up variables

# S = np.array(Sd)

# # adjustments
# S[:, 1] *= -1
# K_eq[1] *= -1

# S[S == -0] = 0
# S_s = -np.copy(S)
# S_p = np.copy(S) #reverse neg sign
# S_s[S > 0] = 0
# S_p[S < 0] = 0

# S_s_nz = np.array(S_s.nonzero())
# S_p_nz = np.array(S_p.nonzero())

# # first coordinate, e.g. metabolites w nonzero substrate/product coeff across all reactions. also works as substrate indices. 
# met_s_nz = S_s_nz[0, :]
# met_p_nz = S_p_nz[0, :]

# # second coordinate, e.g. reactions indices for those concentrations. works to index substrates as well. 
# rxn_s_nz = S_s_nz[1, :]   
# rxn_p_nz = S_p_nz[1, :]

# # one dim is always 2
# n_Km_s = np.max(met_s_nz.shape) 
# n_Km_p = np.max(met_p_nz.shape)

# # c = np.ones(n_met)
# # Km_s = np.random.uniform(0.9, 1.1, n_Km_s)
# # Km_p = np.random.uniform(0.9, 1.1, n_Km_p)
# # cfwd = np.random.uniform(1.8, 2.2, n_rxn)
# # crev = np.zeros(n_rxn)

# c = c.value
# Km_s = Km_s.value
# Km_p = Km_p.value
# cfwd = cfwd.value
# crev = crev.value

# # define Km positions by nonzero S matrix concentrations
# y_s = c[met_s_nz] - Km_s
# y_p = c[met_p_nz] - Km_p

# # index 
# met_s_nz

c0 = c.value[0]

In [772]:
c.value[0] = c0

In [773]:

c.value[0] += 0

In [774]:
sat_expr = []
fwd_sat = []
sat = []

for i in range(n_rxn):
    # sum terms are separate in logsumexp. one per saturation term (row in C_alpha, C_beta)
    n_term_s = np.sum(d_alpha == i) 
    n_term_p = np.sum(d_beta == i)
    n_term = n_term_s + n_term_p
    
    
    Km_s_idx = np.nonzero(S_s_nz[1, :] == i)
    S_s_idx = S_s_nz[0, S_s_nz[1, :] == i] # negate -1 entries
    
    Km_p_idx = np.nonzero(S_p_nz[1, :] == i)
    S_p_idx = S_p_nz[0, S_p_nz[1, :] == i]
    
    #S_s_idx = S_s_nz[0, S_s_nz[1, :] == i]
    
    sat_expr.append(           [ (C_alpha @ y_s.value)[d_alpha == i] ,  
                                 (C_beta @ y_p.value)[d_beta == i],
                                 0,
                                 #-1*np.ones(n_lse_terms - n_term + 1) 
                               ]
                   )
    fwd_sat.append(-S.T[i, S_s_idx] @ y_s.value[Km_s_idx]) # + cfwd.value[i]
    
    

for i, rxn in enumerate(sat_expr):
    s = 0
    
    for term in rxn:
        s += np.sum(np.exp(term))
        
    sat.append(s)

In [779]:
sat

[14446.53749878086, 2804.9557454998526, 428.0954608367081, 2.475230740274676]

In [776]:
fwd_sat

[5.692914348929424,
 7.9366263813070335,
 4.916108782190473,
 -4.5834652320808774e-05]

In [780]:
np.exp(fwd_expr)/np.array(sat)

ValueError: operands could not be broadcast together with shapes (8,) (4,) 

In [526]:
fwd_expr

[16.678073586287503, 9.351914151491968, 10.05749968982474, 2.2002765482147923]

In [ ]:
# est is obj. 
print(np.array([np.exp(-est),np.exp(lvE)]))

Calculate reverse ckat, check if rxns are running forward.

These will be the measured fluxes. Good. Now we can add saturation. Remember that this has to be fixed in pre-estimation, as we cannot just change one param without violating Haldane or equilibrium constraints.

All slightly below our threshold, which is what we wanted. No unboundedness.

# Testing toy problem with thermodynamic and kinetic data

In [43]:
Sd = pd.DataFrame(stoich_dict, dtype=np.int8).fillna(0).astype(np.int8)

n_met = len(Sd.index)
n_rxn = len(Sd.columns)

Sd

,6PFRUCTPHOS-RXN,F16ALDOLASE-RXN,2TRANSKETO-RXN,TRIOSEPISOMERIZATION-RXN
FRUCTOSE-6P,-1,0,1,0
ATP,-1,0,0,0
PROTON,1,0,0,0
ADP,1,0,0,0
FRUCTOSE-16-DIPHOSPHATE,1,-1,0,0
DIHYDROXY-ACETONE-PHOSPHATE,0,1,0,1
GAP,0,1,1,-1
ERYTHROSE-4P,0,0,-1,0
XYLULOSE-5-PHOSPHATE,0,0,-1,0


# Import sim output

In [ ]:
time = '10'
date = '2023-02-13'
experiment = 'balance'
entry = f'{experiment}_{time}_{date}'
folder = f'out/fbagd/{entry}/'

In [ ]:
output = np.load(folder + 'output.npy',allow_pickle='TRUE').item()
# output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
output = output['agents']['0']
fba = output['listeners']['fba_results']
mass = output['listeners']['mass']
bulk = pd.DataFrame(output['bulk'])

In [ ]:
f = open(folder + 'agent_processes.pkl', 'rb')
agent_processes = dill.load(f)
f.close()

f = open(folder + 'agent_steps.pkl', 'rb')
agent_steps = dill.load(f)
f.close()

In [ ]:
getattributes(agent_steps['ecoli-metabolism'])

In [ ]:
s_matrix = agent_steps['ecoli-metabolism'].model.network.s_matrix
stoichiometry = agent_steps['ecoli-metabolism'].stoichiometry

In [ ]:
getmembers(agent_steps['ecoli-metabolism'])[7][1].keys()

In [ ]:
agent_processes

# Check FBA output for central glycolysis

In [ ]:
stoichiometry